In [ ]:
!pip install tensorflow numpy matplotlib scikit-learn

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
train_data_dir = '/content/drive/MyDrive/Kaggle3/Train_Alphabet/'
test_data_dir = '/content/drive/MyDrive/Kaggle3/Test_Alphabet/'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Using 20% of data for validation

train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Replace with your dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    test_data_dir,  # Replace with your dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')


Found 19441 images belonging to 27 classes.
Found 540 images belonging to 27 classes.


In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

Epoch 1/10
292/608 [=============>................] - ETA: 1:04:14 - loss: 0.3040 - accuracy: 0.9164

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


In [ ]:
# Save the model
model.save('/content/drive/MyDrive/Kaggle3/mobilenet_model.h5')


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Get true labels and predicted labels
true_labels = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Predict on validation data
predictions = model.predict(validation_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Classification report
print('Classification Report')
print(classification_report(true_labels, predicted_labels, target_names=class_labels))

# Confusion matrix
print('Confusion Matrix')
print(confusion_matrix(true_labels, predicted_labels))
